# Import Library

In [ ]:
import csv 
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import cv2
import glob
import os
import re
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Ground Truth data by yolov8x-pose

In [ ]:
#root directory
rootDir = r"C:\Users\kawaw\data_Tracking\jointsTracking\1129"
#left
yoloFile_left = os.path.join(rootDir,"yolo_left_GT.csv")
dfGT_left = pd.read_csv(filepath_or_buffer=yoloFile_left)
dataGT_left = dfGT_left.values
#right
yoloFile_right = os.path.join(rootDir,"yolo_right_GT.csv")
dfGT_right = pd.read_csv(filepath_or_buffer=yoloFile_right)
dataGT_right = dfGT_right.values

In [ ]:
print(dataGT_left[0])
print(dataGT_right[0])

## Calculate 3D position

In [ ]:
#constant setting
cameraMatrix = [[297.0, 0, 151.5], # fx: focal length in x, cx: principal point x
                [0, 297.5, 149.0], #fy: focal length in y, cy: principal point y
                [0, 0, 1]          #1: scaling factor
                ]
fX = cameraMatrix[0][0]
fY = cameraMatrix[1][1]
fSkew = cameraMatrix[0][1]
oX = cameraMatrix[0][2]
oY = cameraMatrix[1][2]

transform_cam2base = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]
BASELINE = 280
epsiron = 0.001

In [ ]:
print(dataGT_left.shape)

In [ ]:
def calculate3D(left,right):
    frameIndex = left[0]
    results = []
    for i in range(6):
        xl = left[3*i+1]
        yl = left[3*i+2]
        xr = right[3*i+1]
        yr = right[3*i+2]
        disparity = xl-xr
        if disparity < 0.00001:
            disparity += epsiron
        X = (BASELINE / disparity) * (xl - oX - (fSkew / fY) * (yl - oY))
        Y = (BASELINE * (fX / fY) * (yl - oY) / disparity)
        Z = (fX * BASELINE / disparity)
        X = transform_cam2base[0][0] * X + transform_cam2base[0][1] * Y + transform_cam2base[0][2] * Z + transform_cam2base[0][3]
        Y = transform_cam2base[1][0] * X + transform_cam2base[1][1] * Y + transform_cam2base[1][2] * Z + transform_cam2base[1][3]
        Z = transform_cam2base[2][0] * X + transform_cam2base[2][1] * Y + transform_cam2base[2][2] * Z + transform_cam2base[2][3]
        results.append(frameIndex)
        results.append(X)
        results.append(Y)
        results.append(Z)
    return results

def convert3D(data_left,data_right):
    counter_left = 0
    counter_right = 0
    counter = 1
    results = []
    while (counter<data_right.shape[0]):
        if (data_left[counter_left][0] == data_right[counter_right][0]):
            result = calculate3D(data_left[counter_left],data_right[counter_right])
            results.append(result)
            counter_left+=1
            counter_right+=1
        else:
            if (len(results)>=1):
                result = results[-1]
                for i in range(6):
                    result[3*i] = counter
                results.append(result)
        if (data_left[counter_left][0] < data_right[counter_right][0]):
            counter_left+=1
        elif (data_left[counter_left][0] > data_right[counter_right][0]):
            counter_right += 1
        counter+=1
    return np.array(results)

In [ ]:
pos3D_GT = convert3D(data_left=dataGT_left,data_right=dataGT_right)

### plot 3d position

In [ ]:
markers=['o','s','D','+','x','^','v','>','<','p','H','8']

def plot_data(dataGT,title):
    leftShoulder = dataGT[:,:4]
    rightShoulder = dataGT[:,4:8]
    leftElbow = dataGT[:,8:12]
    rightElbow = dataGT[:,12:16]
    leftWrist = dataGT[:,16:20]
    rightWrist = dataGT[:,20:24]
    fig,ax = plt.subplots(3,1,figsize=(12,8))
    ax[0].plot(leftShoulder[:,0],leftShoulder[:,1],color="r",linestyle="-",marker='o',label="left shoulder")
    ax[1].plot(leftShoulder[:,0],leftShoulder[:,2],color="r",linestyle="-",marker='o')
    ax[2].plot(leftShoulder[:,0],leftShoulder[:,3],color="r",linestyle="-",marker='o')
    ax[0].plot(rightShoulder[:,0],rightShoulder[:,1],color="r",linestyle="--",marker='D',label="right shoulder")
    ax[1].plot(rightShoulder[:,0],rightShoulder[:,2],color="r",linestyle="--",marker='D')
    ax[2].plot(rightShoulder[:,0],rightShoulder[:,3],color="r",linestyle="--",marker='D')
    ax[0].plot(leftElbow[:,0],leftElbow[:,1],color="g",linestyle="-",marker='o',label="left elbow")
    ax[1].plot(leftElbow[:,0],leftElbow[:,2],color="g",linestyle="-",marker='o')
    ax[2].plot(leftElbow[:,0],leftElbow[:,3],color="g",linestyle="-",marker='o')
    ax[0].plot(rightElbow[:,0],rightElbow[:,1],color="g",linestyle="--",marker='D',label="right elbow")
    ax[1].plot(rightElbow[:,0],rightElbow[:,2],color="g",linestyle="--",marker='D')
    ax[2].plot(rightElbow[:,0],rightElbow[:,3],color="g",linestyle="--",marker='D')
    ax[0].plot(leftWrist[:,0],leftWrist[:,1],color="b",linestyle="-",marker='o',label="left Wrist")
    ax[1].plot(leftWrist[:,0],leftWrist[:,2],color="b",linestyle="-",marker='o')
    ax[2].plot(leftWrist[:,0],leftWrist[:,3],color="b",linestyle="-",marker='o')
    ax[0].plot(rightWrist[:,0],rightWrist[:,1],color="b",linestyle="--",marker='D',label="right Wrist")
    ax[1].plot(rightWrist[:,0],rightWrist[:,2],color="b",linestyle="--",marker='D')
    ax[2].plot(rightWrist[:,0],rightWrist[:,3],color="b",linestyle="--",marker='D')
    ax[0].set_ylim(-500,1000)
    ax[1].set_ylim(-500,1000)
    ax[2].set_ylim(0,2000)
    ax[0].set_ylabel("X")
    ax[1].set_ylabel("Y")
    ax[2].set_ylabel("Z")
    ax[2].set_xlabel("frame index")
    ax[0].set_title(title)
    fig.legend()
    plt.show()

plot_data(pos3D_GT,title="3D")

## Plot ground truth data

In [ ]:
markers=['o','s','D','+','x','^','v','>','<','p','H','8']
colors = ["r","g","b","c","m","y","k"]
names = ["LS","RS","LE","RE","LW","RW"]
connections = [[0,1],[0,2],[1,3],[2,4],[3,5]]

In [ ]:
rootDir = r"C:\Users\kawaw\data_Tracking\jointsTracking\1129"
SaveDir = os.path.join(rootDir,"3D_GT")
def plot3DMap(pos3D_GT,saveDir=SaveDir):
    if not os.path.exists(saveDir):
        os.makedirs(saveDir)
    for j in range(pos3D_GT.shape[0]):
        # Create a 3D plot
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111, projection='3d')

        # Plot the 3D data
        for i in range(6):
            ax.plot(pos3D_GT[j][4*i+1], pos3D_GT[j][4*i+2], pos3D_GT[j][4*i+3], color=colors[i],marker=markers[i],markersize=10,label=names[i])
        for [k,l] in connections:
            ax.plot([pos3D_GT[j][4*k+1],pos3D_GT[j][4*l+1]], [pos3D_GT[j][4*k+2],pos3D_GT[j][4*l+2]], [pos3D_GT[j][4*k+3],pos3D_GT[j][4*l+3]], linestyle='-', color='k')

        # Customize the plot
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_xlim(-500,1000)
        ax.set_ylim(-500,1000)
        ax.set_zlim(0,2000)
        ax.set_title(f'{(j+1):03d} frames')
        ax.legend()
        savePath = os.path.join(saveDir,f'{(j+1):03d}.jpg')
        fig.savefig(savePath,bbox_inches='tight')

def atoi(text):
        return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

def makeVideo(srcDir,videoFps,outputVideoName):
        img_array=[]
        for filename in sorted(glob.glob(f"{srcDir}/*.jpg"), key=natural_keys):
            img = cv2.imread(filename)
            height, width, layers = img.shape
            size = (width, height)
            img_array.append(img)

        out = cv2.VideoWriter(outputVideoName, cv2.VideoWriter_fourcc(*'mp4v'), videoFps, size)

        for i in range(len(img_array)):
            out.write(img_array[i])
        out.release()

### make video

In [ ]:
VideoName = os.path.join(rootDir,"3D_GT.mp4")
plot3DMap(pos3D_GT=pos3D_GT,saveDir=SaveDir)
makeVideo(srcDir=SaveDir,videoFps=30,outputVideoName=VideoName)

# Real-time

## load data

In [ ]:
#root directory
rootDir = r"C:\Users\kawaw\data_Tracking\jointsTracking\1129"
#left
file3d_RT = os.path.join(rootDir,"triangulation.csv")
df3d_RT = pd.read_csv(filepath_or_buffer=file3d_RT)
data3d_RT = df3d_RT.values
print(data3d_RT[0])

## plot data

In [ ]:
plot_data(data3d_RT,title="3D")

## plot 3d map

In [ ]:
rootDir = r"C:\Users\kawaw\data_Tracking\jointsTracking\1129"
SaveDir = os.path.join(rootDir,"3D_RT")
VideoName = os.path.join(rootDir,"3D_RT.mp4")
plot3DMap(pos3D_GT=data3d_RT,saveDir=SaveDir)
makeVideo(srcDir=SaveDir,videoFps=30,outputVideoName=VideoName)

# Plot 3D map both GT and Real-time

In [ ]:
markers=['o','s','D','+','x','^','v','>','<','p','H','8']
colors = ["r","g","b","c","m","y","k"]
names = ["LS","RS","LE","RE","LW","RW"]
connections = [[0,1],[0,2],[1,3],[2,4],[3,5]]

In [ ]:
data3d_RT[:,0]

In [ ]:
indexes = np.where(data3d_RT[:,0]==19)[0]
print(indexes[0])

In [ ]:
#plot part
def plotBoth3DMap(pos3D_GT,pos3D_RT,saveDir=SaveDir):
    if not os.path.exists(saveDir):
        os.makedirs(saveDir)
    latestPositions_GT = np.zeros((24))
    latestPositions_RT = np.zeros((24))
    counter = 1
    while counter<pos3D_RT[-1][0] or counter<pos3D_GT[-1][0]:
        # Create a 3D plot
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111, projection='3d')
        #new position available
        if counter in pos3D_RT[:,0]:
            indexes = np.where(pos3D_RT[:,0]==counter)[0]
            if len(indexes) >=2:
                index = indexes[-1]
            else:
                index = indexes[0]
            print(counter, index)
            latestPositions_RT = pos3D_RT[index]
        if counter in pos3D_GT[:,0]:
            indexes = np.where(pos3D_GT[:,0]==counter)[0]
            if len(indexes) >=2:
                index = indexes[-1]
            else:
                index = indexes[0]
            print(counter,index)
            latestPositions_GT = pos3D_GT[index]

        # Plot the 3D data points
        for i in range(6):
            if latestPositions_GT[0] != 0:
                ax.plot(latestPositions_GT[4*i+1], latestPositions_GT[4*i+2], latestPositions_GT[4*i+3], color=colors[0],marker=markers[i],markersize=10,label=names[i])
            if latestPositions_RT[0] != 0:
                ax.plot(latestPositions_RT[4*i+1], latestPositions_RT[4*i+2], latestPositions_RT[4*i+3], color=colors[2],marker=markers[i],markersize=10,label=names[i])
        # connect dots
        for [k,l] in connections:
            if latestPositions_GT[0] != 0:
                ax.plot([latestPositions_GT[4*k+1],latestPositions_GT[4*l+1]], [latestPositions_GT[4*k+2],latestPositions_GT[4*l+2]], [latestPositions_GT[4*k+3],latestPositions_GT[4*l+3]], linestyle='--', color=colors[0])
            if latestPositions_RT[0] != 0:
                ax.plot([latestPositions_RT[4*k+1],latestPositions_RT[4*l+1]], [latestPositions_RT[4*k+2],latestPositions_RT[4*l+2]], [latestPositions_RT[4*k+3],latestPositions_RT[4*l+3]], linestyle='--', color=colors[2])

        # Customize the plot
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_xlim(-500,1000)
        ax.set_ylim(-500,1000)
        ax.set_zlim(0,2000)
        ax.set_title(f'{(counter):03d} frames')
        ax.legend()
        savePath = os.path.join(saveDir,f'{(counter):03d}.jpg')
        fig.savefig(savePath,bbox_inches='tight')
        counter += 1

In [ ]:
rootDir = r"C:\Users\kawaw\data_Tracking\jointsTracking\1129"
SaveDir = os.path.join(rootDir,"3D_RT_GT")
VideoName = os.path.join(rootDir,"3D_RT_GT.mp4")
plotBoth3DMap(pos3D_GT=pos3D_GT,pos3D_RT=data3d_RT,saveDir=SaveDir)
makeVideo(srcDir=SaveDir,videoFps=10,outputVideoName=VideoName)

# Make movie

## Define function 

### Constant value

In [ ]:
#ROI size
ROISIZE=30
COLOR_GT = [255,255,255]
COLOR_RT = [255,0,0]

In [ ]:
class Video():
    def __init__(self,dataGT,videoFps,videoPath,saveImgDir,outputVideoName,sizeRoi=ROISIZE,colorGT=COLOR_GT,colorRT=COLOR_RT):
        self.sizeRoi =sizeRoi
        self.dataGT = dataGT
        self.videoFps = videoFps
        self.videoPath = videoPath
        self.saveImgDir = saveImgDir
        self.outputVideoName = outputVideoName
        self.colorGT = colorGT
        self.colorRT=colorRT
        #draw rectangle in images
        self.editVideo()
        #make video from images
        self.makeVideo()

    def drawRectangle(self,frame,centerX,centerY,color,sizeRoi=ROISIZE,width=320,height=320):
        cv2.rectangle(frame,(min(max(int(centerX-ROISIZE/2),0),width),min(max(int(centerY-ROISIZE/2),0),height)),(max(min(int(centerX+ROISIZE/2),width),0),max(min(int(centerY+ROISIZE/2),height),0)),thickness=2,color=color)

    def editVideo(self):
        #arrange data
        leftShoulderGT = self.dataGT[:,:3]
        rightShoulderGT = self.dataGT[:,3:6]
        leftElbowGT = self.dataGT[:,6:9]
        rightElbowGT = self.dataGT[:,9:12]
        leftWristGT = self.dataGT[:,12:15]
        rightWristGT = self.dataGT[:,15:18]

        # Open the video file
        cap = cv2.VideoCapture(self.videoPath)

        # Check if the video file opened successfully
        if not cap.isOpened():
            print("Error opening video file")
            return

        # Create the output folder if it doesn't exist
        if not os.path.exists(self.saveImgDir):
            os.makedirs(self.saveImgDir)

        # Get the frames per second (fps) and frame dimensions
        #fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Start reading and saving frames
        counter = 1
        while True:
            print(counter)
            ret, frame = cap.read()

            if not ret:
                break
            
            #groundTruth
            #LS
            if counter in leftShoulderGT[:,0]: #detected
                [indexLS] = np.where(leftShoulderGT[:,0]==counter)[0]
                centerX = leftShoulderGT[indexLS,1]
                centerY = leftShoulderGT[indexLS,2]
                self.drawRectangle(frame,centerX=centerX,centerY=centerY,color=self.colorGT,width=width,height=height)
            #RS
            if counter in rightShoulderGT[:,0]:
                [indexRS] = np.where(rightShoulderGT[:,0]==counter)[0]
                [centerX,centerY] = rightShoulderGT[indexRS,1:]
                self.drawRectangle(frame,centerX=centerX,centerY=centerY,color=self.colorGT,width=width,height=height)
            #LE
            if counter in leftElbowGT[:,0]:
                [indexLE] = np.where(leftElbowGT[:,0]==counter)[0]
                [centerX,centerY] = leftElbowGT[indexLE,1:]
                self.drawRectangle(frame,centerX=centerX,centerY=centerY,color=self.colorGT,width=width,height=height)
            #RE
            if counter in rightElbowGT[:,0]:
                [indexRE] = np.where(rightElbowGT[:,0]==counter)[0]
                [centerX,centerY] = rightElbowGT[indexRE,1:]
                self.drawRectangle(frame,centerX=centerX,centerY=centerY,color=self.colorGT,width=width,height=height)
            #LW
            if counter in leftWristGT[:,0]:
                [indexLW] = np.where(leftWristGT[:,0]==counter)[0]
                [centerX,centerY] = leftWristGT[indexLW,1:]
                self.drawRectangle(frame,centerX=centerX,centerY=centerY,color=self.colorGT,width=width,height=height)
            #RW
            if counter in rightWristGT[:,0]:
                [indexRW] = np.where(rightWristGT[:,0]==counter)[0]
                [centerX,centerY] = rightWristGT[indexRW,1:]
                self.drawRectangle(frame,centerX=centerX,centerY=centerY,color=self.colorGT,width=width,height=height)
                
            # Save the frame as an image
            frame_filename = os.path.join(self.saveImgDir, f"{counter:03d}.png")
            cv2.imwrite(frame_filename, frame)

            counter += 1
        # Release the video capture object
        cap.release()

    def atoi(self,text):
        return int(text) if text.isdigit() else text

    def natural_keys(self,text):
        return [ self.atoi(c) for c in re.split(r'(\d+)', text) ]

    def makeVideo(self):
        img_array=[]
        for filename in sorted(glob.glob(f"{self.saveImgDir}/*.png"), key=self.natural_keys):
            img = cv2.imread(filename)
            height, width, layers = img.shape
            size = (width, height)
            img_array.append(img)

        out = cv2.VideoWriter(self.outputVideoName, cv2.VideoWriter_fourcc(*'mp4v'), self.videoFps, (320,320))

        for i in range(len(img_array)):
            out.write(img_array[i])
        out.release()

In [ ]:
CAMERA = "right"
rootDir = r"C:\Users\kawaw\data_Tracking\jointsTracking\1129"
VideoPath = os.path.join(rootDir,"joints_front_{}.mp4".format(CAMERA))
SaveImgDir = os.path.join(rootDir,"imgs_{}".format(CAMERA))
OutputVideoName = os.path.join(rootDir,"joints_front_{}_GT.mp4".format(CAMERA))
video = Video(dataGT=dataGT_right,videoFps=30,videoPath=VideoPath,saveImgDir=SaveImgDir,outputVideoName=OutputVideoName,sizeRoi=ROISIZE,colorGT=COLOR_GT,colorRT=COLOR_RT)